In [2]:
import string

import numpy as np
import cupy as cp

# Cuda Device Info

In [3]:
print('Detected devices:', cp.cuda.runtime.getDeviceCount())
print('Device info:', cp.cuda.runtime.getDeviceProperties(0))

Detected devices: 1
Device info: {'name': b'GeForce GTX 1660 SUPER', 'totalGlobalMem': 6230114304, 'sharedMemPerBlock': 49152, 'regsPerBlock': 65536, 'warpSize': 32, 'maxThreadsPerBlock': 1024, 'maxThreadsDim': (1024, 1024, 64), 'maxGridSize': (2147483647, 65535, 65535), 'clockRate': 1800000, 'totalConstMem': 65536, 'major': 7, 'minor': 5, 'textureAlignment': 512, 'texturePitchAlignment': 32, 'multiProcessorCount': 22, 'kernelExecTimeoutEnabled': 1, 'integrated': 0, 'canMapHostMemory': 1, 'computeMode': 0, 'maxTexture1D': 131072, 'maxTexture2D': (131072, 65536), 'maxTexture3D': (16384, 16384, 16384), 'concurrentKernels': 1, 'ECCEnabled': 0, 'pciBusID': 1, 'pciDeviceID': 0, 'pciDomainID': 0, 'tccDriver': 0, 'memoryClockRate': 7001000, 'memoryBusWidth': 192, 'l2CacheSize': 1572864, 'maxThreadsPerMultiProcessor': 1024, 'isMultiGpuBoard': 0, 'cooperativeLaunch': 1, 'cooperativeMultiDeviceLaunch': 1, 'deviceOverlap': 1, 'maxTexture1DMipmap': 32768, 'maxTexture1DLinear': 268435456, 'maxTextu

# Creating the cipher alphabet

The alpabeth must have a prime length. 

In [9]:
alphabet = dict((i, letter) for i, letter in enumerate("_" + string.ascii_letters + string.digits + '.' + '?' + '!' + ','))
inv_alphabet = dict((letter, i) for i, letter in enumerate("_" + string.ascii_letters + string.digits + '.' + '?' + '!' + ','))
alphabet_codes = [key for key in alphabet]
mod = len(alphabet)
print('Alpabeth length:', len(alphabet))

Alpabeth length: 67


# Pick a random key

In [48]:
l = 6

while True:
    key = cp.random.choice(alphabet_codes, (l, l))
    det = cp.linalg.det(key).round().astype(cp.int64) % mod
    
    if det != 0 and cp.gcd(det, len(alphabet)) == 1:
        break
key

array([[66,  1,  3, 21, 26,  3],
       [32, 50, 22, 37,  2,  3],
       [ 4, 23,  1, 37,  4, 44],
       [57, 36, 55,  6, 21, 45],
       [12, 40, 60, 52, 24, 44],
       [19, 28, 28,  4, 43, 42]])

# Get user message


In [49]:
user_msg = input('Enter a message:')
preprocessed_msg = user_msg.strip().replace(" ", "_")
msg = np.array([inv_alphabet.get(char, np.nan) for char in preprocessed_msg])

if np.isnan(np.sum(msg)):
    print('An invalid character was detected.')
elif msg.shape[0] < key.shape[0]:
    print(f'Invalid length. {msg.shape[0]} < {key.shape[0]}')
else:
    original_msg_len = len(msg)
    msg.resize((key.shape[0], 1 + msg.shape[0] - key.shape[0]))
    msg = cp.array(msg)
    print(msg)

Enter a message: vitor oliveira


[[22  9 20 15 18  0 15 12  9]
 [22  5  9 18  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]


# Encryption

- $K$ represents the $n \times n$ Key matrix.
- $M$ represents the $n \times k$ Message matrix, which can be padded with $0$s.

The cipher text, $C$, is generated with the following formula: $C = K \cdot M$

In [50]:
encrypted_block = key.dot(msg) % mod
encrypted_msg = encrypted_block.ravel().tolist()
print(encrypted_block, ''.join([alphabet[char] for char in encrypted_msg]), sep='\n')

[[ 0 63 56  3 50  0 52 55 58]
 [62  2 18 40 23  0 11 49 20]
 [58 17 19  5 28  0 60 48 36]
 [36 23 57 29 57  0 51 14 44]
 [ 5 40 64 29 55  0 46 10 41]
 [29 43 29 52 35  0 17 27 37]]
_.3cX_Z259brNw_kWt5qseB_7VJJw4C4_YnReN?C2_TjOCQCZI_qAK


# Decryption

- $K^{-1}$ represents the inverted Key matrix. It can be generated with the following formula: $K^{-1} = d^{-1} \times adj(K)$, where $d^{-1} = mmi(det(K), m) \mod m$ is the Modular multiplicative inverse of $K$ determinant module by the alphabet length, $m$, and $adj(K)$ is the $K$ adjugate matrix. 

The decrypted text, $P$, is generated with the following formula: $P = K^{-1} \cdot C$

<!-- [inverse key matrix](https://crypto.interactive-maths.com/hill-cipher.html) -->

In [83]:
# https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/
# https://stackoverflow.com/a/6528024

def modular_power(x, y, m):
    if y == 0:
        return 1
 
    p = modular_power(x, y // 2, m) % m
    p = (p * p) % m
 
    if (y & 1)  == 0:
        return p
    return ((x * p) % m)

def mmi(a, m):
    g = cp.gcd(a, m)
    
    if g != 1:
        return -1
    
    return modular_power(a, m - 2, m)

def inverse_matrix(matrix, m):
    det = cp.linalg.det(matrix).round().astype(cp.int64) % m
    inv_det = mmi(det, m)
    
    return inv_det * (cp.linalg.inv(matrix.astype(cp.float64)) * cp.linalg.det(matrix) % m)

In [84]:
inverse_key = inverse_matrix(key, mod)
decrypted_block = inverse_key.dot(encrypted_block).round().astype(cp.int64) % mod
decrypted_msg = decrypted_block.ravel().tolist()

if len(decrypted_msg) > original_msg_len:
    diff = len(decrypted_msg) - original_msg_len
    decrypted_msg = decrypted_msg[:-diff]

try:
    assert(cp.equal(decrypted_block, msg).all() == True)
    print(decrypted_block, ''.join([alphabet[char] for char in decrypted_msg]), sep='\n')
except AssertionError:
    print('Decryption failed.')

[[22  9 20 15 18  0 15 12  9]
 [22  5  9 18  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]
vitor_oliveira
